In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate

2025-03-24 12:17:02.549885: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-24 12:17:02.669605: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-24 12:17:02.725331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742818622.769757   75840 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742818622.812154   75840 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742818622.999306   75840 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
df = pd.read_csv('/workspaces/Spam-Email-Classifier/data/spam.csv')

In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df = df[['Category', 'Message']]

In [5]:
df.columns = ['label', 'message']

In [6]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df['message'], df['label'], test_size=0.2, random_state=42)

In [9]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

LogisticRegression Model

In [10]:
log_reg = LogisticRegression()
log_reg.fit(X_train_tfidf, y_train)
y_pred_log_reg = log_reg.predict(X_test_tfidf)

In [11]:
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_log_reg))
print("Classification Report:\n", classification_report(y_test, y_pred_log_reg))

Logistic Regression Accuracy: 0.9739910313901345
Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.99       966
           1       1.00      0.81      0.89       149

    accuracy                           0.97      1115
   macro avg       0.99      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



SVM Model

In [12]:
svm = SVC(kernel='linear',probability=True)

In [13]:
svm.fit(X_train_tfidf, y_train)

SVC(kernel='linear', probability=True)

In [14]:
y_pred_svm = svm.predict(X_test_tfidf)

In [15]:
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print("Classification Report:\n", classification_report(y_test, y_pred_svm))

SVM Accuracy: 0.9910313901345291
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       966
           1       0.99      0.94      0.97       149

    accuracy                           0.99      1115
   macro avg       0.99      0.97      0.98      1115
weighted avg       0.99      0.99      0.99      1115



MLP Scikit learn Model

In [16]:
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=42)

In [17]:
mlp.fit(X_train_tfidf, y_train)

MLPClassifier(max_iter=300, random_state=42)

In [18]:
y_pred_mlp = mlp.predict(X_test_tfidf)

In [19]:
print("MLP Accuracy:", accuracy_score(y_test, y_pred_mlp))
print("Classification Report:\n", classification_report(y_test, y_pred_mlp))

MLP Accuracy: 0.9937219730941704
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       966
           1       1.00      0.95      0.98       149

    accuracy                           0.99      1115
   macro avg       1.00      0.98      0.99      1115
weighted avg       0.99      0.99      0.99      1115



Hybrid Model Using Tensorflow

In [20]:
train_preds = np.column_stack([
    log_reg.predict_proba(X_train_tfidf)[:, 1],
    svm.predict_proba(X_train_tfidf)[:, 1],
    mlp.predict_proba(X_train_tfidf)[:, 1]
])

In [21]:
test_preds = np.column_stack([
    log_reg.predict_proba(X_test_tfidf)[:, 1],
    svm.predict_proba(X_test_tfidf)[:, 1],
    mlp.predict_proba(X_test_tfidf)[:, 1]
])

In [22]:
input_layer = Input(shape=(train_preds.shape[1],))
dense_layer = Dense(3, activation='relu')(input_layer)
output_layer = Dense(1, activation='sigmoid')(dense_layer)

2025-03-24 12:17:54.995579: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [23]:
meta_model = Model(inputs=input_layer, outputs=output_layer)
meta_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
meta_model.fit(train_preds, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


112/112 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8629 - loss: 0.6268 - val_accuracy: 0.9966 - val_loss: 0.5391
Epoch 2/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9972 - loss: 0.5174 - val_accuracy: 0.9978 - val_loss: 0.4388
Epoch 3/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9976 - loss: 0.4144 - val_accuracy: 0.9978 - val_loss: 0.3481
Epoch 4/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9973 - loss: 0.3278 - val_accuracy: 0.9978 - val_loss: 0.2714
Epoch 5/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9949 - loss: 0.2598 - val_accuracy: 0.9978 - val_loss: 0.2102
Epoch 6/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9953 - loss: 0.2004 - val_accuracy: 0.9978 - val_loss: 0.1630
Epoch 7/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9962 - loss: 0.1566 - val_accuracy: 0.9989 - val_loss: 0.1274
Epoch 8/10
112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9971 - loss: 0.1220 - val_accuracy: 0.9989 - val_

In [25]:
y_pred_meta = (meta_model.predict(test_preds) > 0.5).astype(int)

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [26]:
print("Stacked Model Accuracy:", accuracy_score(y_test, y_pred_meta))
print("Classification Report:\n", classification_report(y_test, y_pred_meta))

Stacked Model Accuracy: 0.9928251121076234
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       966
           1       0.99      0.95      0.97       149

    accuracy                           0.99      1115
   macro avg       0.99      0.98      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [29]:
import pickle

In [31]:
with open('log_reg.pkl', 'wb') as file:
    pickle.dump(log_reg, file)
    
with open('svm.pkl', 'wb') as file:
    pickle.dump(svm, file)
    
with open('mlp.pkl', 'wb') as file:
    pickle.dump(mlp, file)

In [35]:
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf_vectorizer, f)

In [32]:
meta_model.save("stacked_model.keras")

Testing the hybrid model

In [33]:
def predict_spam(message):
    #transform the recieved message into tfidf form
    message_tfidf = tfidf_vectorizer.transform([message])

    #get the predictions from base model for metamodel input
    stacked_input = np.column_stack([
        log_reg.predict_proba(message_tfidf)[:, 1],
        svm.predict_proba(message_tfidf)[:, 1],
        mlp.predict_proba(message_tfidf)[:, 1]
    ])
    
    
    final_prediction = (meta_model.predict(stacked_input) > 0.5).astype(int)

    return "Spam" if final_prediction[0][0] == 1 else "Ham"

In [34]:
test_messages = [
    "Win a brand new car! Click here to enter the lucky draw now!",
    "Hey, just checking in. Are you free this weekend?",
    "Congratulations! You've been selected for a special prize. Claim now!",
    "Your account has been suspended due to unusual activity. Verify now!",
    "Let's catch up over coffee sometime next week.",
    "Limited-time offer! Get 50% off on all electronics. Shop now!",
    "Reminder: Your doctor's appointment is scheduled for tomorrow at 10 AM.",
    "Exclusive deal for you! Buy one, get one free. Offer expires soon.",
    "Are you available for a quick meeting this afternoon?",
    "You've won a $500 Amazon gift card! Claim your reward here.",
    "Dear customer, your loan application has been approved.",
    "Hey, did you complete the project? Let me know if you need help.",
    "Your Netflix subscription is about to expire. Renew now to continue watching.",
    "URGENT: Your PayPal account has been compromised. Reset your password now.",
    "Join us for a free webinar on data science this weekend!",
    "Your OTP for login is 123456. Do not share it with anyone.",
    "This is a final notice regarding your car's extended warranty.",
    "A new message from your bank: Your transaction of $1000 was successful.",
    "FREE vacation to the Bahamas! Click now to book your trip.",
    "Don't forget our dinner plan at 7 PM tonight. See you there!"
]

for i, msg in enumerate(test_messages):
    print(f"Test {i+1}: {msg}")
    print("Prediction:", predict_spam(msg))
    print("-" * 50)


Test 1: Win a brand new car! Click here to enter the lucky draw now!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


Prediction: Spam
--------------------------------------------------
Test 2: Hey, just checking in. Are you free this weekend?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: Ham
--------------------------------------------------
Test 3: Congratulations! You've been selected for a special prize. Claim now!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: Spam
--------------------------------------------------
Test 4: Your account has been suspended due to unusual activity. Verify now!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Prediction: Ham
--------------------------------------------------
Test 5: Let's catch up over coffee sometime next week.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Prediction: Ham
--------------------------------------------------
Test 6: Limited-time offer! Get 50% off on all electronics. Shop now!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Prediction: Spam
--------------------------------------------------
Test 7: Reminder: Your doctor's appointment is scheduled for tomorr